In [ ]:
import pandas as pd

In [ ]:
cs_churn=cs[cs['이탈여부']=='이탈']
cs_non = cs[cs['이탈여부']=='유지']

# 이상치 비율 확인
# 1분위
q1_churn = cs_churn[cs_churn['이탈여부']=='이탈'][TARGET].quantile(q=0.25)
# 3분위
q3_churn = cs_churn[cs_churn['이탈여부']=='이탈'][TARGET].quantile(q=0.75)
IQR =  q3_churn - q1_churn

q1 = cs_non[[TARGET]].quantile(q=0.25)
q3 = cs_non[[TARGET]].quantile(q=0.75)